# NIH Chest X-ray Multi label Binary classification using Tensorflow Densenet121 (Transfer learning)


In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd


'/workspaces/chest-x-ray-diagnosis'

## Imports

In [2]:
# Set environment variables
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

2025-03-20 11:19:05.357058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742469545.370087  113993 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742469545.374203  113993 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
found_gpu = tf.config.list_physical_devices('GPU')
if not found_gpu:
     raise Exception("No GPU found")
found_gpu, tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import opendatasets as od

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


In [5]:
%matplotlib inline

# auto reload libs
%load_ext autoreload
%autoreload 2

## Download the dataset


In [6]:
dataset_url = 'https://www.kaggle.com/datasets/nih-chest-xrays/sample'

# Look into the data directory
datasets = 'datasets/sample'
dataset_path = Path(datasets)
IMAGE_DIR = dataset_path /'sample/images'
CSV_PATH = dataset_path /'sample/sample_labels.csv'
dataset_path.mkdir(parents=True, exist_ok=True)
if not dataset_path.is_dir():
    od.download(dataset_url)

## Paths Setup

In [7]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_IMAGES_DIR)

datasets/sample/sample/images


## Constants

In [8]:
IMAGE_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE
NUM_EPOCHS = cfg.TRAIN.NUM_EPOCHS
LEARNING_RATE = cfg.TRAIN.LEARNING_RATE

## Load Datasets

In [9]:
# drop_colums = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
#        'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding',
#        'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
# 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',

# labels =['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
#        'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
#        'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
CLASSES_NAME = ['Atelectasis','Effusion','Infiltration', 'Mass', 'No Finding']#,'Nodule']

In [11]:
from src.data_loader.chest_x_ray_preprocessor import ChestXRayPreprocessor

preprocessor = ChestXRayPreprocessor(cfg, labels=CLASSES_NAME)
train_ds, valid_ds, pos_weights, neg_weights, steps_per_epoch = preprocessor.get_training_and_validation_datasets()

2025-03-20 11:21:38,761 — src.data_loader.chest_x_ray_preprocessor — INFO — Getting training and validation datasets with batch size:None
2025-03-20 11:21:38,764 — src.data_loader.chest_x_ray_preprocessor — INFO — Loading and preprocessing dataframe
2025-03-20 11:21:38,772 — src.data_loader.chest_x_ray_preprocessor — INFO — Cleaning up training dataframe
2025-03-20 11:21:38,800 — src.data_loader.chest_x_ray_preprocessor — INFO — Loaded dataframe with shape: (5606, 2) and 5606 rows
2025-03-20 11:21:38,823 — src.data_loader.chest_x_ray_preprocessor — INFO — training split: 3947 and validation split: 987


2025-03-20 11:21:39,481 — src.data_loader.chest_x_ray_preprocessor — INFO — Normalization layer adapted
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2025-03-20 11:21:39,921 — src.data_loader.chest_x_ray_preprocessor — INFO — Normalizing image
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2025-03-20 11:21:40,813 — src.data_loader.chest_x_ray_preprocessor — INFO — Preparing training dataset with augmentation
2025

In [12]:
for batch in train_ds.take(1):
    images, labels = batch
    print(images.shape, labels.shape)
    print(images[0].shape, images[0].numpy().min(), images[0].numpy().max(), labels[0])
    print(labels[:10])

(16, 350, 350, 3) (16, 5)
(350, 350, 3) -2.1179042 2.0638473 tf.Tensor([0. 0. 0. 0. 1.], shape=(5,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]], shape=(10, 5), dtype=float32)


In [13]:
for batch in valid_ds.take(1):
    images, labels = batch
    print(images.shape, labels.shape)
    print(images[0].shape, images[0].numpy().min(), images[0].numpy().max(), labels[0])
    print(labels[:10])

(16, 350, 350, 3) (16, 5)
(350, 350, 3) -2.117904 2.64 tf.Tensor([0. 0. 0. 0. 1.], shape=(5,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]], shape=(10, 5), dtype=float32)


In [ ]:
def plot_random_images(train_ds, num_images=9):
  """
  Plots a random sample of images and their corresponding labels from a TensorFlow dataset.

  Args:
    train_ds: A TensorFlow dataset object containing image-label pairs.
    num_images: The number of images to plot (default: 9).
  """

  # Ensure the dataset is shuffled (if it's not already)
  train_ds = train_ds.shuffle(buffer_size=1024)

  # Take a batch of images and labels
  _images = []
  _labels = []
  for images, labels in train_ds:
      for image, label in zip(images, labels):
        _images.append(image)
        _labels.append(label)
        if len(images) >= num_images:
            break
        
      break

  # Create a figure and axes for the plot
  plt.figure(figsize=(10, 10))

  # Iterate through the images and plot them
  for i in range(num_images):
    ax = plt.subplot(int(num_images**0.5), int(num_images**0.5), i + 1) # Create a grid of subplots
    plt.imshow(images[i].numpy().astype("uint8"), cmap='gray') # Convert to numpy and uint8 for display
    plt.title(f"Label: {labels[i].numpy()}") # Display the label
    plt.axis("off") # Hide the axes

  plt.tight_layout()
  plt.show()

In [ ]:
plot_random_images(train_ds)

#### Compute Class Frequencies

In [ ]:
# N = train_cat_labels_df.shape[0]
# positive_frequencies = (train_cat_labels_df==1).sum()/N
# negative_frequencies = (train_cat_labels_df==0).sum()/N
# positive_frequencies, negative_frequencies

In [ ]:
# data_df = pd.DataFrame(list(positive_frequencies.items()), columns=['class', 'positives'])
# data_df['negatives'] = negative_frequencies.values
# data_df

In [ ]:
# data_df.plot.bar(x='class')

As we see in the above plot, the contributions of positive cases is significantly lower than that of the negative ones. However, we want the contributions to be equal. One way of doing this is by multiplying each example from each class by a class-specific weight factor, $w_{pos}$ and $w_{neg}$, so that the overall contribution of each class is the same. 

To have this, we want 

$$w_{pos} \times freq_{p} = w_{neg} \times freq_{n},$$

which we can do simply by taking 

$$w_{pos} = freq_{neg}$$
$$w_{neg} = freq_{pos}$$

This way, we will be balancing the contribution of positive and negative labels.

In [ ]:
# _pos_weights = negative_frequencies.values
# _neg_weights = positive_frequencies.values
# positive_frequencies.values

# Try adjusting the weight balance slightly
# pos_weights = np.sqrt(negative_frequencies.values) * 0.8  # Reduce positive weight slightly
# neg_weights = np.sqrt(positive_frequencies.values) * 1.2  # Increase negative weight slightly

In [ ]:
# pos_contirbution = positive_frequencies * _pos_weights
# neg_contribution = negative_frequencies * _neg_weights

# pos_contirbution, neg_contribution

In [ ]:
# weighted_data_df = pd.DataFrame(list(pos_contirbution.items()), columns=['class', 'positives'])
# weighted_data_df['negatives'] = neg_contribution.values
# weighted_data_df

In [ ]:
# weighted_data_df.plot.bar(x='class')

## Weighted loss calculation to handle class imbalance

As the above figure shows, by applying these weightings the positive and negative labels within each class would have the same aggregate contribution to the loss function. Now let's implement such a loss function. 

After computing the weights, our final weighted loss for each training case will be 

$$\mathcal{L}_{cross-entropy}^{w}(x) = - (w_{p} y \log(f(x)) + w_{n}(1-y) \log( 1 - f(x) ) ).$$

In [ ]:
import keras
@keras.saving.register_keras_serializable()
def _get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)
    
    Returns:
      weighted_loss (function): weighted loss function
    """
    @keras.saving.register_keras_serializable()
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value. 

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (float): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0

        for i in range(len(pos_weights)):
            y = y_true[:, i]
            f_of_x = y_pred[:, i]

            f_of_x_log = K.log(f_of_x + epsilon)
            f_of_x_1_min_log = K.log((1-f_of_x) + epsilon)

            first_term = pos_weights[i] * y * f_of_x_log
            sec_term = neg_weights[i] * (1-y) * f_of_x_1_min_log
            loss_per_col = - K.mean(first_term + sec_term)
            loss += loss_per_col
        return loss

    return weighted_loss



## Model Development

### Load and Prepare DenseNet121 Model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
import mlflow
to_monitor = 'val_AUC'
mode = 'max'
mlflow.start_run()
mlflow.tensorflow.autolog(log_models=True, 
                        log_datasets=False, 
                        log_input_examples=True,
                        log_model_signatures=True,
                        keras_model_kwargs={"save_format": "keras"},
                        checkpoint_monitor=to_monitor, 
                        checkpoint_mode=mode)

In [ ]:
base_model = DenseNet121(
     include_top=False,
     weights=None, #'imagenet', 
     input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)  
)

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(CLASSES_NAME), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
METRICS = [
    'binary_accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC'), 
]

In [ ]:
from src.weighted_loss.weighted_loss import get_weighted_loss  

model.compile(optimizer=tf.keras.optimizers.AdamW(), 
               loss=get_weighted_loss(pos_weights, neg_weights),
        metrics=METRICS)     

model.summary()

## Model training

## Callbacks

In [ ]:
CHECK_POINT_DIR = 'exported_models'
checkpoint_prefix = os.path.join(CHECK_POINT_DIR, "ckpt_{epoch}.keras")
LOG_DIR = 'logs'

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                        save_best_only=True, # Save only the best model based 'to_monitor'
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),  # Display checkpoint saving messages
    tf.keras.callbacks.ReduceLROnPlateau(monitor=to_monitor,
                                        mode=mode, factor=0.1, patience=5, min_lr=1e-7),
    tf.keras.callbacks.EarlyStopping(monitor=to_monitor,
                                        mode=mode, patience=10, restore_best_weights=True),
]

In [ ]:
history = model.fit(train_ds, 
                    validation_data=valid_ds,
                    batch_size=cfg.TRAIN.BATCH_SIZE,
                    epochs = cfg.TRAIN.NUM_EPOCHS,
                    callbacks=callbacks)


In [ ]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec
import numpy as np

# 1. Input Schema
# -----------------
# Your input is a batch of images with shape (32, 240, 240, 3)
# We use -1 to indicate that the batch size can vary.
input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, IMAGE_SIZE, IMAGE_SIZE, 1), "image")])

# 2. Output Schema - Multilabel binary classification head
# ------------------
# Your model outputs a list of two arrays. We need to define a schema for each.
# Array 1: Shape (1, 4)
output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, len(CLASSES_NAME)), "classification")])

# 3. Model Signature
# --------------------
# Combine the input and output schemas into a ModelSignature
signature = ModelSignature(inputs=input_schema, outputs=output_schema)
mlflow.tensorflow.log_model(
    model,
    "best_model",
    signature=signature,
    code_paths=["src"],
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train_loss', 'val_loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])
plt.legend(['AUC', 'val_AUC'])
plt.ylabel("AUC")
plt.xlabel("epoch")
plt.title("Training AUC Curve")
plt.show()

## Model Evaluation

In [ ]:
del train_ds, valid_ds

In [ ]:
from src.data_loader.chest_x_ray_preprocessor import ChestXRayPreprocessor

test_prorcessor  = ChestXRayPreprocessor(cfg, labels=CLASSES_NAME)
test_ds = test_prorcessor.get_test_dataset()

In [ ]:
for batch in test_ds.take(1):
    images, labels = batch
    print(images.shape, labels.shape)
    print(images[0].shape, images[0].numpy().min(), images[0].numpy().max(), labels[0])

In [ ]:
result = model.evaluate(test_ds, return_dict=True, batch_size=BATCH_SIZE)
result

In [ ]:
mlflow.log_metrics(result)
mlflow.end_run()
result

In [ ]:
preds = model.predict(test_ds)
preds = preds.astype(int)

In [ ]:
preds

In [ ]:
tf_model = tf.keras.saving.load_model('exported_models/ckpt_2.keras')
tf_model.summary()

In [ ]:
model_uri = 'runs:/74cb397adcc642459ac5b1cd754cf74c/best_model'

model = mlflow.pyfunc.load_model(model_uri=model_uri)
